In [1]:
# Load the extension, disable it after development
%load_ext autoreload
%autoreload 2

from preprocess import *

import numpy as np
import pandas as pd

from fastdtw import fastdtw
from dtaidistance import dtw
from scipy.spatial.distance import squareform, pdist

import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})


In [3]:
df = pd.read_csv('../data/2019-08-01_test2.csv')
df=df[['ip','timestamp']]

#A = get_aggregated(df, '2019-08-01 00:00:00','2019-08-02 00:00:00')
#A = get_aggregated(df)

tfrom=datetime.datetime.fromisoformat('2019-08-01 00:00:00').timestamp()
tto=datetime.datetime.fromisoformat('2019-08-02 00:00:00').timestamp()

agg_secs = 900
days_proc = 1

A = df.copy()
A = df[df['timestamp']>=tfrom]
A = A[A['timestamp']<tto]

A['timestamp'] = A.timestamp-tfrom
A['timestamp'] = np.floor(A.timestamp/agg_secs)
Ag = A.groupby('ip')['timestamp'].agg([list, 'count'])

#df['agg']=np.floor(df['timestamp']/900)
#exp=df.groupby('agg')['ip'].agg([set])
#sns.barplot(A['timestamp','count'])

data = Ag[Ag['count'] > 10] #prefiltered data
#data = Ag

###


In [4]:
data['series']=data['list'].apply(get_bin_series, args=[np.int((days_proc*24*3600)/agg_secs)])
data['active']=data['series'].apply(np.sum)

data.sort_values('active', inplace=True, ascending=True)
#vect=pd.concat([pd.DataFrame(index=data.index, data=np.stack(data.series)),data['count']], axis=1)

vect=pd.DataFrame(index=data.index, data=np.stack(data.series))
#vect=vect[data['active']<60]

C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

In [5]:
pairwise = pd.DataFrame(squareform(pdist(vect,metric='jaccard')), index=data.index, columns=data.index, dtype=np.float32)

In [6]:
threshold=0.90 #for corelation to be positive at least as much have to match
window_threshold=0.8

for act in range(1,np.int(np.floor(90*window_threshold))):
    subset = pairwise[data['active']>=act-np.ceil(act*(1-window_threshold))]
    subset = subset[data['active']<=act+3]#np.floor(act*(1-window_threshold))]
    
    counts=(pairwise <= 0.05).apply(np.sum, axis=1)
    break

In [7]:
#sns.lineplot(counts)


In [8]:
#labels = hdbscan.HDBSCAN(
#    min_samples=30,
#    min_cluster_size=50,
#    metric='precomputed'
#).fit_predict(subset)

In [9]:
vect.to_csv('../data/2019-08-01_jaccard.csv')